In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from lab_coffee_utils import load_coffee_data 
import logging

/Users/vipulgarg/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
X, Y = load_coffee_data()
print(X.shape, Y.shape)

(200, 2) (200, 1)


In [3]:
print(f"Temperature Max Min pre-normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max Min pre-normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

Temperature Max Min pre-normalization: 284.99, 151.32
Duration    Max Min pre-normalization: 15.45, 11.51
Temperature Max, Min post normalization: 1.66, -1.69
Duration    Max, Min post normalization: 1.79, -1.70


In [4]:
Xt = np.tile(Xn, (1000, 1))
Yt = np.tile(Y, (1000, 1))
print(Xt.shape, Yt.shape) 

(200000, 2) (200000, 1)


In [5]:
tf.random.set_seed(1234)
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(3, activation='sigmoid', name='L1'),
        Dense(1, activation='sigmoid', name='L2')
    ]
)

In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ L1 (Dense)                      │ (None, 3)              │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L2 (Dense)                      │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
L1_num_params = 2 * 3 + 3   # W1 parameters  + b1 parameters
L2_num_params = 3*1 + 1
print(f"L1 params={L1_num_params}, L2 params={L2_num_params}, total Params = {L1_num_params + L2_num_params}")


L1 params=9, L2 params=4, total Params = 13


In [8]:
W1, b1 = model.get_layer("L1").get_weights()
W2, b2 = model.get_layer("L2").get_weights()

print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)


W1(2, 3):
 [[-0.03862381 -0.16481805 -0.3873549 ]
 [ 0.6002706   0.2676587  -0.46439487]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[1.2051908 ]
 [1.153648  ]
 [0.40151322]] 
b2(1,): [0.]


In [9]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)

Epoch 1/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 245us/step - loss: 0.3462
Epoch 2/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 245us/step - loss: 0.3462
Epoch 2/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 249us/step - loss: 0.1494
Epoch 3/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 249us/step - loss: 0.1494
Epoch 3/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 255us/step - loss: 0.1393
Epoch 4/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 255us/step - loss: 0.1393
Epoch 4/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 249us/step - loss: 0.1356
Epoch 5/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 249us/step - loss: 0.1356
Epoch 5/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 1s 238us/step - loss: 0.1336
Epoch 6/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 1s 238us/step - loss: 0.1336
Epoch 6/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 245us/step - loss: 0.1325
Epoch 7/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 245us/step - loss: 0.1325
Epoch 7/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 242us/step - loss: 0.1317
Epoch 8/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 242us/step - loss: 0.1317
Epoch 

In [11]:
W1, b1 = model.get_layer("L1").get_weights()
W2, b2 = model.get_layer("L2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[  8.6304245   6.935896   10.969291 ]
 [  2.6434553 -16.541973    7.9725347]] 
b1: [12.158303  20.028831  -1.3756177]
W2:
 [[  9.715814 ]
 [  7.0559297]
 [-22.814243 ]] 
b2: [-14.575415]


In [13]:
# After finishing the lab later, you can re-run all 
# cells except this one to see if your trained model
# gets the same results.

# Set weights from a previous run. 
W1 = np.array([
    [-8.94,  0.29, 12.89],
    [-0.17, -7.34, 10.79]] )
b1 = np.array([-9.87, -9.28,  1.01])
W2 = np.array([
    [-31.38],
    [-27.86],
    [-32.79]])
b2 = np.array([15.54])

# Replace the weights from your trained model with
# the values above.
model.get_layer("L1").set_weights([W1,b1])
model.get_layer("L2").set_weights([W2,b2])

In [14]:
# Check if the weights are successfully replaced
W1, b1 = model.get_layer("L1").get_weights()
W2, b2 = model.get_layer("L2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[-8.94  0.29 12.89]
 [-0.17 -7.34 10.79]] 
b1: [-9.87 -9.28  1.01]
W2:
 [[-31.38]
 [-27.86]
 [-32.79]] 
b2: [15.54]


In [16]:
X_test = np.array([
    [200,13.9],  # positive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
predictions = 
 [[9.625127e-01]
 [3.031606e-08]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
predictions = 
 [[9.625127e-01]
 [3.031606e-08]]


In [17]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]
